In [1]:
import pandas as pd


In [109]:
df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla.csv")

In [110]:
df.shape[0]

8322

In [111]:
lai = 0.15
tc = 5

df.columns = map(str.lower, df.columns)
if 'fecha' in df.columns:
    df = df.drop('fecha', axis=1)

# change column name 'velocidad'
substrings_to_replace_velocidad = ["vv", "viento", "velocidad", "velocidad del viento"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_velocidad):
        new_column_name = "Velocidad del viento (m/s)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'mp2,5'
substrings_to_replace_mp = ["mp", "2,5", "25"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "MP2,5 (µg/m³)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'altura'
substrings_to_replace_mp = ["mh", "altura", "mezcla"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "Altura de Mezcla (m)"
        df.rename(columns={column: new_column_name}, inplace=True)

df['Velocidad del viento (m/s)'] = df['Velocidad del viento (m/s)'].apply(lambda x: float(str(x).replace(",","."))).round()

table1 = {
    'Velocidad del viento (m/s)': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    'Promedio': [0, 0.03, 0.09, 0.15, 0.17, 0.19, 0.2, 0.56, 0.92, 0.92, 2.11, 2.11, 2.11, 2.11],
    'Mínimo': [0, 0.006, 0.012, 0.018, 0.022, 0.025, 0.029, 0.056, 0.082, 0.082, 0.57, 0.57, 0.57, 0.57],
    'Máximo': [0, 0.042, 0.163, 0.285, 0.349, 0.414, 0.478, 1.506, 2.534, 2.534, 2.534, 2.534, 2.534, 2.534],
    '% Resuspensión': [0, 1.5, 3, 4.5, 6, 7.5, 9, 10, 11, 12, 13, 16, 20, 23]
}
reference_table = pd.DataFrame(table1)

df = pd.merge(df, reference_table, on='Velocidad del viento (m/s)', how='left')

df['LAI (m2/m2)'] = lai

df['Vd,min (cm/s)'] = df['LAI (m2/m2)'] * df['Mínimo']
df['Vd,max (cm/s)'] = df['LAI (m2/m2)'] * df['Máximo']
df['Vd (cm/s)'] = df['LAI (m2/m2)'] * df['Promedio']

flux = pd.DataFrame()
flux['MP2,5 (g/m³)']    = df['MP2,5 (µg/m³)'] / 1000 / 1000
flux["Vd,min (m/s)"]    = df['Vd,min (cm/s)'] / 100
flux["Vd,max (m/s)"]    = df['Vd,max (cm/s)'] / 100
flux["Vd (m/s)"]        = df['Vd (cm/s)'] / 100
flux["fmin,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,min (m/s)"] * 3600
flux["fmax,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,max (m/s)"] * 3600
flux["ft (g/m2*h)"]     = flux['MP2,5 (g/m³)'] * flux["Vd (m/s)"] * 3600

In [113]:
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.16f}'.format

In [119]:
resuspension = pd.DataFrame()
resuspension['% Resuspensión']  = df['% Resuspensión']
resuspension['fmin,t (g/m2*h)'] = flux["fmin,t (g/m2*h)"]
resuspension['fmax,t (g/m2*h)'] = flux["fmax,t (g/m2*h)"]
resuspension['ft (g/m2*h)']     = flux["ft (g/m2*h)"]
resuspension['At min (g/m2*h)'] = 0
resuspension['At max (g/m2*h)'] = 0
resuspension['At (g/m2*h)']     = 0
resuspension['Rmin (g/m2*h)']   = 0
resuspension['Rmax (g/m2*h)']   = 0
resuspension['Rt (g/m2*h)']     = 0

rows = []
# for fila in range(resuspension.shape[0]-1):
for fila in range(2):

    resuspension['Rmin (g/m2*h)']   = ( resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] - resuspension['Rmin (g/m2*h)']
    resuspension['Rmax (g/m2*h)']   = ( resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] - resuspension['Rmax (g/m2*h)']
    resuspension['Rt (g/m2*h)']     = ( resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] ) * resuspension['% Resuspensión'] / 100
    resuspension['At (g/m2*h)']     = resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] - resuspension['Rt (g/m2*h)']

    print(f"""
        fila = {fila}
        resuspension['Rmin (g/m2*h)']   = ( {resuspension['At min (g/m2*h)'].shift(1)} + {resuspension['fmin,t (g/m2*h)'].iloc[-1]} ) * {resuspension['% Resuspensión']} / 100
        resuspension['At min (g/m2*h)'] =   {resuspension['At min (g/m2*h)'].shift(1)} + {resuspension['fmin,t (g/m2*h)']}   - {resuspension['Rmin (g/m2*h)']}
        resuspension['Rmax (g/m2*h)']   = ( {resuspension['At max (g/m2*h)'].shift(1)} + {resuspension['fmax,t (g/m2*h)']} ) * {resuspension['% Resuspensión']} / 100
        resuspension['At max (g/m2*h)'] =   {resuspension['At max (g/m2*h)'].shift(1)} + {resuspension['fmax,t (g/m2*h)']}   - {resuspension['Rmax (g/m2*h)']}
        resuspension['Rt (g/m2*h)']     = ( {resuspension['At (g/m2*h)'].shift(1)}     + {resuspension['ft (g/m2*h)']}     ) * {resuspension['% Resuspensión']} / 100
        resuspension['At (g/m2*h)']     =   {resuspension['At (g/m2*h)'].shift(1)}     + {resuspension['ft (g/m2*h)']}       - {resuspension['Rt (g/m2*h)']}
    """)

    row = {}
    row['Rmin (g/m2*h)']   = resuspension['Rmin (g/m2*h)'].iloc[fila+1]
    row['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].iloc[fila+1]
    row['Rmax (g/m2*h)']   = resuspension['Rmax (g/m2*h)'].iloc[fila+1]
    row['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].iloc[fila+1]
    row['Rt (g/m2*h)']     = resuspension['Rt (g/m2*h)'].iloc[fila+1]
    row['At (g/m2*h)']     = resuspension['At (g/m2*h)'].iloc[fila+1]

    rows.append(row)


        fila = 0
        resuspension['Rmin (g/m2*h)']   = ( 0                     NaN
1                     NaN
2      0.0000006497820000
3      0.0000003394224000
4      0.0000001276560000
              ...        
8317   0.0000011837232000
8318   0.0000011278872000
8319   0.0000010953468000
8320   0.0000011603250000
8321   0.0000006788448000
Name: At min (g/m2*h), Length: 8322, dtype: float64 + 5.702400000000001e-07 ) * 0      4.5000000000000000
1      4.5000000000000000
2      3.0000000000000000
3      1.5000000000000000
4      1.5000000000000000
              ...        
8317   6.0000000000000000
8318   4.5000000000000000
8319   4.5000000000000000
8320   3.0000000000000000
8321   3.0000000000000000
Name: % Resuspensión, Length: 8322, dtype: float64 / 100
        resuspension['At min (g/m2*h)'] =   0                     NaN
1                     NaN
2      0.0000006497820000
3      0.0000003394224000
4      0.0000001276560000
              ...        
8317   0.0000011837232000
831